In [1]:
import pickle
import pandas as pd
import os           
import numpy as np
import re
import itertools
import plotly.express as px
#import brightway2 as bw 
import random 
import string

In [2]:
import sys
sys.path.append('../../Module')  #two levels up & then down to Module folder
from common_mod import *

In [3]:
LCIAname = ["CML", "SP", "openLCA", "BW2"]  #source LCIA name 
lcia_name = "CML_Ozone layer depletion"            # name will be used when saving data/plots
SP_name = "Ozone layer depletion (ODP)"
cml_col_name = "ODP_kgCFC-11"
cml_col_keep = [2,14,15,16,17] # these are "unit",'cas_number','flow'... to keep for each IC, will append cml_col_name later
olca_filepathname = "../../Data_rawCFs/olca_CML_ODP_saved.dat"
bw2_filepathname =  "../../Data_rawCFs/BW2_CML2001_stratospheric ozone depletion.dat"
#used in final calculation to create dict names
pairwise_name = ["SPvsCML", "SPvsBW2", "SPvsopenLCA", "openLCAvsBW2", "openLCAvsCML", "BW2vsCML"]

<h3> read in raw cleaned-up CML and select ODP cols </h3>

In [4]:
cml = pd.read_excel("CML_cleaned_11IC.xlsx" )  #header=[1,2,3]
cml.columns

Index(['Unnamed: 0', 'Substance', 'unit', 'ADPelement_kgSb', 'ADPfossil_MJ',
       'GWP_kgCO2', 'ODP_kgCFC-11', 'HTP_kg1,4DCB', 'FAETP_kg1,4DCB',
       'MAETP_kg1,4DCB', 'TETP_kg1,4DCB', 'POFP_kgEthylene', 'AP_kgSO2',
       'EP_kgPO4', 'cas_number', 'flow', 'subcategory', 'category'],
      dtype='object')

In [5]:
cml_col_keep.append(cml.columns.get_loc(cml_col_name) )
cml_col_keep

[2, 14, 15, 16, 17, 6]

In [6]:
cml_raw = cml.iloc[: , cml_col_keep]
print(len(cml_raw))
cml_raw = cml_raw.dropna(subset=[cml_col_name])  #drop all na value for the IC
print(len(cml_raw))
cml_raw = cml_raw.rename(columns={cml_col_name:"value"  })
cml_raw = cml_raw.rename(str.lower, axis='columns')
#cml_raw.head()

1961
24


In [7]:
cml_raw['cas_number'] = add_rand_str_toCAS(cml_raw["cas_number"])
cml_raw['flow'] = cml_raw['flow'].str.lower()
cml_raw.head()

,unit,cas_number,flow,subcategory,category,value
105,kg,000071-55-6,"1,1,1-trichloroethane",unspecified,air,0.12
422,kg,000075-69-4,cfc-11,unspecified,air,1.00
423,kg,000076-13-1,cfc-113,unspecified,air,1.00
424,kg,000076-14-2,cfc-114,unspecified,air,0.94
425,kg,000076-15-3,cfc-115,unspecified,air,0.44


<h3> read in raw SP </h3>

In [8]:
SP_cml = pd.read_excel("../../Data_rawCFs/SP_CML-IA.xlsx", sheet_name=SP_name, 
                                  header=[0], converters={'CAS':str})

print(len(SP_cml))
#SP_cml.columns
SP_cml = SP_cml.rename(columns={"Compartment": "category", "Subcompartment": "subcategory", 
                               "Substance": "flow", "Factor":"value", "CAS": "cas_number" })
SP_cml = SP_cml.rename(str.lower, axis='columns')

23


In [9]:
SP_cml["cas_number"] = add_rand_str_toCAS(SP_cml["cas_number"])
SP_cml['flow'] = SP_cml['flow'].str.lower()
SP_cml.head()

,category,subcategory,flow,cas_number,value,unit
0,Air,(unspecified),"ethane, 1-chloro-1,1-difluoro-, hcfc-142b",000075-68-3,0.07,kg CFC-11 eq / kg
1,Air,(unspecified),"ethane, 1,1-dichloro-1-fluoro-, hcfc-141b",001717-00-6,0.12,kg CFC-11 eq / kg
2,Air,(unspecified),"ethane, 1,1,1-trichloro-, hcfc-140",000071-55-6,0.12,kg CFC-11 eq / kg
3,Air,(unspecified),"ethane, 1,1,1,2-tetrafluoro-2-bromo-, halon 2401",000124-72-1,0.25,kg CFC-11 eq / kg
4,Air,(unspecified),"ethane, 1,1,2-trichloro-1,2,2-trifluoro-, cfc-113",000076-13-1,1.00,kg CFC-11 eq / kg


<h3> read in raw openLCA dat </h3>

In [10]:
olca_cml = unpickle_data(olca_filepathname)
olca_cml = olca_cml.rename(str.lower, axis='columns')
print(len(olca_cml))
#print(olca_cml.columns)

122


In [11]:
olca_cml["cas_number"] = add_rand_str_toCAS(olca_cml["cas_number"])
olca_cml['flow'] = olca_cml['flow'].str.lower()
olca_cml.head()

,impact_method,impact_category_uuid,impact_category,reference_unit,flow_uuid,flow,cas_number,subcategory,category,unit,value
0,CML-IA baseline,44c2d0e0-0745-3f3e-bffb-bfaea092e3c4,Ozone layer depletion (ODP),kg CFC-11 eq,3b30d058-02ef-4ed2-b357-5fc1a45c63e7,"ethane, 1,2-dichloro-1,1,2,2-tetrafluoro-, cfc...",000076-14-2,high population density,Emission to air,kg,0.94
1,CML-IA baseline,44c2d0e0-0745-3f3e-bffb-bfaea092e3c4,Ozone layer depletion (ODP),kg CFC-11 eq,27f61651-96a5-45c7-9873-832626cf6905,"ethane, 2-chloro-1,1,1,2-tetrafluoro-, hcfc-124",002837-89-0,high population density,Emission to air,kg,0.02
2,CML-IA baseline,44c2d0e0-0745-3f3e-bffb-bfaea092e3c4,Ozone layer depletion (ODP),kg CFC-11 eq,db907ff7-6cf0-4f88-9376-116262dfad6d,"ethane, 2,2-dichloro-1,1,1-trifluoro-, hcfc-123",000306-83-2,high population density,Emission to air,kg,0.02
3,CML-IA baseline,44c2d0e0-0745-3f3e-bffb-bfaea092e3c4,Ozone layer depletion (ODP),kg CFC-11 eq,33a69662-583b-45b8-8cb9-94f59fca9133,"methane, trichlorofluoro-, cfc-11",TPM5PR2G0QGN3,high population density,Emission to air,kg,1.00
4,CML-IA baseline,44c2d0e0-0745-3f3e-bffb-bfaea092e3c4,Ozone layer depletion (ODP),kg CFC-11 eq,3613c452-7500-4753-819d-4a310943ce48,"propane, 1,3-dichloro-1,1,2,2,3-pentafluoro-, ...",000507-55-1,high population density,Emission to air,kg,0.03


<h3> read in BW2 </h3>

In [12]:
bw_cml = unpickle_data(bw2_filepathname)
print(len(bw_cml))
bw_cml["cas_number"] = add_rand_str_toCAS(bw_cml["cas_number"])
bw_cml['flow'] = bw_cml['flow'].str.lower()
bw_cml.head()

80


,cas_number,flow,flow_uuid,type,category,subcategory,unit,value
0,000071-55-6,"ethane, 1,1,1-trichloro-, hcfc-140",f8ee4881-a003-4e18-aa2a-d9daf97d76f7,emission,air,"low population density, long-term",kilogram,0.12
1,000071-55-6,"ethane, 1,1,1-trichloro-, hcfc-140",9e7d019f-c8c0-4631-9bbb-ced3e081c90d,emission,air,lower stratosphere + upper troposphere,kilogram,0.12
2,000071-55-6,"ethane, 1,1,1-trichloro-, hcfc-140",99585564-bfce-4845-9aaa-2f24b8f26a41,emission,air,non-urban air or from high stacks,kilogram,0.12
3,000071-55-6,"ethane, 1,1,1-trichloro-, hcfc-140",ce6294f5-2ed7-46ee-a967-33e265e34455,emission,air,unspecified,kilogram,0.12
4,000071-55-6,"ethane, 1,1,1-trichloro-, hcfc-140",818cee9e-231c-4b53-8ed2-47a0001802d5,emission,air,urban air close to ground,kilogram,0.12


### 1. after preparing the data, see different emission compartment/category by diff. source, need to convert to common_catg

In [13]:
all_raw_LCIAdf = [cml_raw, SP_cml, olca_cml, bw_cml]

In [14]:
uniq_catg_all_raw_LCIAdf = []
for lcia in all_raw_LCIAdf: 
    uniq_catg_all_raw_LCIAdf.append( uniq_catg(lcia))

In [15]:
#uniq_catg_all_raw_LCIAdf
print([uniq_catg_all_raw_LCIAdf[i][0] for i in range(len(uniq_catg_all_raw_LCIAdf))])

[1, 1, 6, 5]


<h3> for each LCIA source, checking for a same EF within a same common_category, if duplicated values </h3>
<h4> it is common for a same EF with diff. CF value under different emission compartment, but within one compartment, there shouldn't be any duplications</h4>

<h4> <font color = 'red'> after checking by either flow name or cas_number, if no duplicated CF found for a same EF, can be used for pairwise comp. </font> </h4>

<font color = 'cyan'> checking by flow name </font>

In [16]:
check_EF_allLCIA = []
for n in range(len(all_raw_LCIAdf)): 
    df = change_source_catg(all_raw_LCIAdf[n], source = LCIAname[n],cat = "category", sub_cat = "subcategory")
    print ("Start checking for ", LCIAname[n])
    check_EF_allLCIA.append(check_EF_value_oneLCIA(df, "common_category", "flow", "value") )

Start checking for  CML
Checking by flow same EF does not have different CFs under: emission/air/unspecified , ok to be used.
Start checking for  SP
Checking by flow same EF does not have different CFs under: emission/air/unspecified , ok to be used.
Start checking for  openLCA
Checking by flow same EF does not have different CFs under: emission/air/urban , ok to be used.
Checking by flow same EF does not have different CFs under: emission/air/rural , ok to be used.
Checking by flow same EF does not have different CFs under: emission/air/rural-LT , ok to be used.
Checking by flow same EF does not have different CFs under: emission/air/unspecified , ok to be used.
Start checking for  BW2
Checking by flow same EF does not have different CFs under: emission/air/rural-LT , ok to be used.
Checking by flow same EF does not have different CFs under: emission/air/rural , ok to be used.
Checking by flow same EF does not have different CFs under: emission/air/unspecified , ok to be used.
Checkin

<font color = 'cyan'> checking by flow CAS </font>

In [17]:
check_EF_allLCIA = []
for n in range(len(all_raw_LCIAdf)):
    df = change_source_catg(all_raw_LCIAdf[n], source = LCIAname[n], cat = "category", sub_cat = "subcategory")
    print ("Start checking for ", LCIAname[n])
    check_EF_allLCIA.append(check_EF_value_oneLCIA(df, "common_category", "cas_number", "value") )

Start checking for  CML
Checking by cas_number same EF does not have different CFs under: emission/air/unspecified , ok to be used.
Start checking for  SP
Checking by cas_number same EF does not have different CFs under: emission/air/unspecified , ok to be used.
Start checking for  openLCA
Checking by cas_number same EF does not have different CFs under: emission/air/urban , ok to be used.
Checking by cas_number same EF does not have different CFs under: emission/air/rural , ok to be used.
Checking by cas_number same EF does not have different CFs under: emission/air/rural-LT , ok to be used.
Checking by cas_number same EF does not have different CFs under: emission/air/unspecified , ok to be used.
Start checking for  BW2
Checking by cas_number same EF does not have different CFs under: emission/air/rural-LT , ok to be used.
Checking by cas_number same EF does not have different CFs under: emission/air/rural , ok to be used.
Checking by cas_number same EF does not have different CFs un

### 2. extract intersection of EFs from two source LCIA for each common_category for pair-wise comparison of CFs and re-combine EFs from different category to one final DF
run together within functions() in next step

when converting each source_category to common_category, need to specify the LCIA source: <font color='red'> source = "openLCA", "SP", "BW2", "CML" </font>

### 3. final calculation, pair-wise comparison, corr. matrix

In [18]:
# create 6 empty dict: dict_SPvsRIVM; dict_SPvsBW2; dict_SPvsopenLCA; dict_openLCAvsBW2; dict_openLCAvsRIVM; dict_BW2vsRIVM
# each dict with five keys and null values (to be assigned later)
for name in pairwise_name:
    keys = [name+"_commonEF_"+ name.split("vs",1)[0], name+"_commonEF_"+ name.split("vs",1)[1],
                 name+"_commonEF"+ "_sumtable", name + "_result_corr", name + "_result_diffEFs" ]
    globals()['dict_%s' % name] = dict.fromkeys(keys, None)

<h4> <font color="cyan"> Assign values to the first three keys for each dict:  commonEFs for the 2 pair-wise LCIA and its sum_table </font> </h4>
below are dict_names, each dict use its corresponding raw LCIA DF and LCIA_names used in final_2concat_DF() function
<li> dict_SPvsCML <font color='red'> comp. by  CAS </font> </li>
<li> dict_SPvsBW2 <font color='red'> comp. by  CAS  </font> </li>
<li> dict_SPvsopenLCA <font color='red'> comp. by  CAS  </font> </li>
<li> dict_openLCAvsBW2 <font color='red'> comp. by flow_UUID </font> </li> 
<li> dict_openLCAvsCML <font color='red'> comp. by CAS  </font> </li>
<li> dict_BW2vsCML <font color='red'> comp. by CAS  </font> </li>

In [27]:
k_list = list(dict_SPvsCML)
dict_SPvsCML[k_list[0]],dict_SPvsCML[k_list[1]],dict_SPvsCML[k_list[2]] = final_2concat_DF(SP_cml, "SP", cml_raw, "CML")
#                                        "flow", "category", "subcategory", "flow", "category", "subcategory", "common_category")

1 unique common categories are extracted: ['emission/air/unspecified']


In [28]:
k_list = list(dict_SPvsBW2)
dict_SPvsBW2[k_list[0]],dict_SPvsBW2[k_list[1]],dict_SPvsBW2[k_list[2]] = final_2concat_DF(SP_cml, "SP", bw_cml, "BW2")
#                                        "flow", "category", "subcategory", "flow", "category", "subcategory", "common_category")

1 unique common categories are extracted: ['emission/air/unspecified']


In [29]:
k_list = list(dict_SPvsopenLCA)
dict_SPvsopenLCA[k_list[0]],dict_SPvsopenLCA[k_list[1]],dict_SPvsopenLCA[k_list[2]] = final_2concat_DF(SP_cml, "SP", olca_cml, "openLCA")
#                                       "flow", "category", "subcategory", "flow", "category", "subcategory", "common_category")

1 unique common categories are extracted: ['emission/air/unspecified']


In [30]:
k_list = list(dict_openLCAvsBW2)
dict_openLCAvsBW2[k_list[0]],dict_openLCAvsBW2[k_list[1]],dict_openLCAvsBW2[k_list[2]] = final_2concat_DF(olca_cml, "openLCA", bw_cml, "BW2", 
                                        "flow_uuid", "category", "subcategory", "flow_uuid", "category", "subcategory", "common_category")

4 unique common categories are extracted: ['emission/air/rural-LT', 'emission/air/urban', 'emission/air/rural', 'emission/air/unspecified']


In [31]:
k_list = list(dict_openLCAvsCML)
dict_openLCAvsCML[k_list[0]],dict_openLCAvsCML[k_list[1]],dict_openLCAvsCML[k_list[2]] = final_2concat_DF(olca_cml, "openLCA", cml_raw, "CML")
#                                        "flow", "category", "subcategory", "flow", "category", "subcategory", "common_category")

1 unique common categories are extracted: ['emission/air/unspecified']


In [32]:
k_list = list(dict_BW2vsCML)
dict_BW2vsCML[k_list[0]],dict_BW2vsCML[k_list[1]],dict_BW2vsCML[k_list[2]] = final_2concat_DF(bw_cml,"BW2", cml_raw, "CML")
#                                       "flow", "category", "subcategory", "flow", "category", "subcategory", "common_category")

1 unique common categories are extracted: ['emission/air/unspecified']


<h4> <font color="cyan"> Assign values to 4th and 5th keys for each dict: result_corr_coeff, and those EFs with diff. CFs </font> </h4>

In [33]:
# assign values to keys for each dict: 2. assign result_corr_coeff , and those EFs with diff. CFs:
for name in pairwise_name:  # dict[item]=value
    pairwise1 = globals()['dict_%s' % name][name+"_commonEF_"+ name.split("vs",1)[0]]
    pairwise2 = globals()['dict_%s' % name][name+"_commonEF_"+ name.split("vs",1)[1]]
    globals()['dict_%s' % name][name + "_result_corr"] = final_pairwise_CF_corr_spearman(pairwise1, pairwise2, "value")
    globals()['dict_%s' % name][name + "_result_diffEFs"] = final_pairwise_save_diff_EF(pairwise1, pairwise2, "value")

In [34]:
# create a new DF saving number_of_common_categories, total_N_of_commonEFs, resulting pairwise correlation coeff.,
CorrEffdict = {}
for name in pairwise_name:
    # the final corr. coeff
    final_corr = globals()['dict_%s' % name][name + "_result_corr"] 
    # to extract number_of_common_categories (e,g, air/rural), and total_N_of_commonEFs used for pairwise comp.
    total_commonEFs = 0
    common_cat = 0
    # each dict['_commonEF_sumtable'] is a list of pd.DF, len is N of common_cat, sum all common_EFs from each common_cat
    for sum_table in globals()['dict_%s' % name][name + "_commonEF_sumtable"]: #each sum_table is a pd.DF for a common_cat
        sumEF = sum_table["EF_in_both"].sum(axis=0)
        common_cat += 1
        total_commonEFs += sumEF
    #  to extract number of EFs with diff. CFs 
    N_diffEFs = len(globals()['dict_%s' % name][name + "_result_diffEFs"])
    # update the dict: keys are names, values have 4 variable:
    CorrEffdict.update({name: [common_cat, total_commonEFs, final_corr, N_diffEFs]}) 

# save to pd.DF
CorrEff_df = pd.DataFrame.from_dict (CorrEffdict, orient='index', columns=["N_common_cat", "Total_N_common_EFs",'result_CorrCoeff', "result_N_diffEFs"])

CorrEff_df

,N_common_cat,Total_N_common_EFs,result_CorrCoeff,result_N_diffEFs
SPvsCML,1,23.0,1.0,0
SPvsBW2,1,16.0,1.0,0
SPvsopenLCA,1,16.0,1.0,0
openLCAvsBW2,4,62.0,1.0,0
openLCAvsCML,1,16.0,1.0,0
BW2vsCML,1,16.0,1.0,0


In [35]:
if not os.path.exists("results/Emissions_IC/result_CorrCoeff"):
    os.mkdir("results/Emissions_IC/result_CorrCoeff")
filename = lcia_name + "_pairwise_comp_CorrEoeff.xlsx"
writer = pd.ExcelWriter("results/Emissions_IC/result_CorrCoeff/" + filename, engine='xlsxwriter')
CorrEff_df.to_excel(writer, sheet_name='Corr_coeff')
writer.save()

<h4><font color='cyan'> Saving result_for each pairwise comp, EFs with different CFs </font></h4>

In [36]:
# saving the pairwise result_EFs with different CFs, to sub-folder "result_diff_EFs_data/"
if not os.path.exists("results/Emissions_IC/result_diff_EFs_data"):
    os.mkdir("results/Emissions_IC/result_diff_EFs_data")
filename = lcia_name + "_pairwise_comp_diff_EFs.xlsx"
writer = pd.ExcelWriter("results/Emissions_IC/result_diff_EFs_data/" + filename, engine='xlsxwriter')
for name in pairwise_name:
    diff_EF = globals()['dict_%s' % name][name + "_result_diffEFs"]
    # Write each dataframe to a different worksheet, sheet_name is the pairwise_name
    diff_EF.to_excel(writer, sheet_name=name)
writer.save()

<h3> To plot out all EFs with diff. CFs, saved to plotly.HTML </h3>

In [38]:
for name in pairwise_name:
    name1, name2 = name.split("vs",1)[0], name.split("vs",1)[1]
    diff_EF = globals()['dict_%s' % name][name + "_result_diffEFs"]
    fig = final_diff_EF_plot2(lcia_name, name1, name2, diff_EF, "flow", "value", "CFvalue_Method2" )
    
    # to save HTML figures: 
    if not os.path.exists("results/Emissions_IC/diff_EFs_HTMLimage"):
        os.makedirs("results/Emissions_IC/diff_EFs_HTMLimage")
    filename = lcia_name + "_" + name1 + "_divided_by_" + name2 + ".html"
    fig.write_html("results/Emissions_IC/diff_EFs_HTMLimage/" + filename)

In [39]:
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

dd = pd.DataFrame(CorrEff_df["result_CorrCoeff"])
dd = dd.sort_values(by="result_CorrCoeff")

fig = make_subplots(rows=1, cols=2, subplot_titles=("Corr_Coeff_heatmap", 
                                 "N of EFs used for calc and resulting EFs w/h diff. CFs"), column_widths=[0.2, 0.8])

fig1 = ff.create_annotated_heatmap(x=dd.columns.to_list(), y=dd.index.to_list(), z=dd.values, hoverinfo='z',
                                   showscale=False)  #showscale=True will override barplot legend

fig.add_trace(fig1.data[0], 1, 1)


trace1 = go.Bar(x=CorrEff_df.index.to_list(), y= CorrEff_df["Total_N_common_EFs"].values, 
                name="Total_N_common_EFs")
trace2 = go.Bar(x=CorrEff_df.index.to_list(), y= CorrEff_df["result_N_diffEFs"].values, 
               name = "result_N_EFs_wh_diffCFs")

fig.add_traces([trace1, trace2], 1, 2)

fig.layout.update({'width':1000})

fig.show()